# Map Layer Update: EXAMPLE

This notebook serves as template and guidance for some common updates on map layers: tile cache, decode function, legend and other descriptions...  

Here, these tasks are performed using [LMIpy](https://lmipy.readthedocs.io/en/latest/), the tool developed by **Vizzuality** to interact with [Resource Watch API](http://api.resourcewatch.org/) assets. These actions and changes could also be done by other API interaction methods, such as Python `requests` library or Postman



### Load libraries

In [1]:
import os
import json
import geopandas as gpd
from IPython.display import display
import getpass
import requests

In [2]:
# Install LMIpy if needed
#!pip install LMIPy
#!python setup.py develop 

from IPython.display import clear_output
clear_output()

import LMIPy as lmi
from LMIPy import utils, dataAPI

print(f'LMI ver. {lmi.__version__} ready!')

LMI ver. 0.6.2 ready!


## Authentiction  
To modify Resource Watch API's assets, the user needs a RW API account ([see here on creation](https://resource-watch.github.io/doc-api/quickstart.html#2-create-an-account-with-the-rw-api)) with editor privileges, and use it to retrieve the API token key.

In [3]:
email = getpass.getpass('Login email:')
password = getpass.getpass('Login password:')

In [4]:
payload = {
    "email": f"{email}",
    "password": f"{password}"
}

url = f'https://api.resourcewatch.org/auth/login'

headers = {'Content-Type': 'application/json'}

r = requests.post(url, data=json.dumps(payload), headers=headers)

API_TOKEN = r.json().get('data').get('token')

## Get dataset of interest  
Datasets can contain one or more layers, and included higher-level attributes. It is suggested to first retrieve the dataset containing the layer to modify, and then access the layer.  

To find the desired dataset, we search it using LMIpy and defining:  
- a keyword to search  
- the app it belongs to (gfw in this case)  
- the type of asset (dataset, layer or both)  
- the environment (staging, preproduction or production)

**Find staging dataset**

In [5]:
search_term = 'loss'
app = ['gfw']
env = 'staging'
#object_type = ['layer', 'dataset']
object_type = 'dataset' #search for dataset only
collection = lmi.Collection(search=search_term, app=app, env=env, object_type=object_type)
collection

[0. Dataset 4820ad38-92d3-46b7-88e4-323306e720c6 Tree cover loss 2020 (LM v3),
 1. Dataset 30ec7d3d-1047-4e11-a765-785cead8375a Tree cover loss 2021 (LM v3) STAGING,
 2. Dataset 5ee44ec5-1d02-4521-b16d-693f75710bbb Tree cover loss by dominant driver (LM v3) STAGING,
 3. Dataset 1e24c6d9-6735-45ad-b2d3-53419af35eb3 Tree cover loss by dominant driver 2020 (LM v3),
 4. Dataset e3df0fca-e230-40c1-9fac-d7ad40df2b31 Tree cover loss due to fires (v202205) STAGING,
 5. Dataset 79bc3d81-2326-4513-9d4f-3ed6f5003231 [TEST] Carbon dioxide emissions from tree cover loss 2020 (LM v3)

**Check production datasets for reference**

In [6]:
#search_term = 'loss'
#app = ['gfw']
env = 'production'
#object_type = ['layer', 'dataset']
object_type = 'dataset'
collection_prod = lmi.Collection(search=search_term, app=app, env=env, object_type=object_type)
collection_prod

[0. Dataset f6c67028-d820-46f3-9190-97cc56e7fef0 Carbon dioxide emissions from tree cover loss (LM v3),
 1. Dataset 39e02031-5214-4ff1-806f-23f4485ab2ff Carbon dioxide emissions from tree cover loss in drained peat (LM v3),
 2. Dataset 7acca489-9398-416e-a877-944a2075d9e4 Tree Cover Loss 2018 Change - Geostore - v20200222,
 3. Table 857b515e-2b36-41b5-a58c-d4e7202e254a Tree Cover Loss 2018 Summary - GADM Adm1 level - v20190701,
 4. Table 82a317da-1fec-4bd0-b9d6-ebe786b9f269 Tree Cover Loss 2018 Summary - GADM Adm2 level - v20190701,
 5. Table 10e124f0-3703-41e3-8136-492b131d5326 Tree Cover Loss 2018 Summary - GADM ISO level - v20190701,
 6. Dataset 2aea608a-acee-4605-ac58-0245796f7c45 Tree Cover Loss 2018 Summary - minimal - geostore - latest,
 7. Dataset 78408179-1fec-47d3-b3d0-f2dbd71f0ab0 Tree Cover Loss 2020 Change - GADM Adm1 level - v20210323,
 8. Dataset 1649e720-9305-408d-bb91-6e22f9039cd4 Tree Cover Loss 2020 Change - GADM Adm2 level - v20210323,
 9. Dataset 2a22d84a-721c-41a4-9de5-19890342f3c1 Tree Cover Loss 2020 Change - GADM Iso level - v20210323,
 10. Dataset 29758f3d-c4d0-4a82-a15b-76aa6036e94c Tree Cover Loss 2020 Change - Geostore - v20210323,
 11. Dataset 06ac26c2-035d-4c85-9cc1-31db29d7d797 Tree Cover Loss 2020 Change - Geostore - v20210323,
 12. Dataset 12c4d1c3-3c8a-4296-ba42-8c430b6b31d3 Tree Cover Loss 2020 Change - Geostore - v20210330,
 13. Dataset 8a4332da-0d5b-4ac3-9a1b-4fe101daecb3 Tree Cover Loss 2020 Summary - GADM Adm1 level - v20210323,
 14. Dataset 1754999b-3093-4eaa-a6b9-f63da2471dc1 Tree Cover Loss 2020 Summary - GADM Adm2 level - v20210218,
 15. Dataset c9142f45-4fec-47a3-a03c-0078b3015118 Tree Cover Loss 2020 Summary - GADM Adm2 level - v20210323,
 16. Dataset a19d146d-bead-4dfe-a4cb-8fa0e9793b5e Tree Cover Loss 2020 Summary - GADM Iso level - v20210323,
 17. Dataset 84aaf340-1a46-46a6-ba84-be1e2a5b5d68 Tree Cover Loss 2020 Whitelist - GADM Adm1 level - v20210323,
 18. Dataset 0190dadb-2f53-415e-95c8-aba1b10afdfa Tree Cover Loss 2020 Whitelist - GADM Adm2 level - v20210323,
 19. Dataset d050aa07-c823-4736-9549-394e7532c0b1 Tree Cover Loss 2020 Whitelist - GADM Iso level - v20210323,
 20. Table cdc5217b-09b7-461d-961d-dc262ba2b4be Tree cover loss 2018 - GADMv3.6 ADM1 summary - v20190423,
 21. Table bd42375f-0983-4e4f-9602-806eb2c26401 Tree cover loss 2018 - GADMv3.6 ADM2 summary - v20190423,
 22. Table f56a1761-d6be-40ec-9cd3-df16d3588480 Tree cover loss 2018 - GADMv3.6 ADM2 summary - v20190429,
 23. Table 97546f05-3dce-4dd0-9abf-80fd1bff9cee Tree cover loss 2018 - GADMv3.6 ISO summary - v20190423,
 24. Dataset b584954c-0d8d-40c6-859c-f3fdf3c2c5df Tree cover loss 2019 (LM v3),
 25. Dataset 893d2c15-7f3c-4fbc-bc84-2bedfd863e84 Tree cover loss 2021 (LM v3),
 26. Dataset f23a1803-ba17-4fe3-a1ad-39abfbfb56c6 Tree cover loss by dominant driver (LM v3),
 27. Dataset 051df3a6-3428-410d-bda0-fc905ac031ad Tree cover loss due to fires (v202205)(DEPRECATED),
 28. Dataset bd66ee37-ad8d-4c6a-b6ab-0294ecd1759d Tree cover loss due to fires (v202207)

In [8]:
ds_prod = collection_prod[25] #get index from list
#prod_example = lmi.Dataset(id_hash = ) #alternative if we already have the dataset id
ds_prod

Dataset 893d2c15-7f3c-4fbc-bc84-2bedfd863e84 Tree cover loss 2021 (LM v3)

## Check dataset properties

In [9]:
ds =  collection[1] #get index from list
ds

Dataset 30ec7d3d-1047-4e11-a765-785cead8375a Tree cover loss 2021 (LM v3) STAGING

In [10]:
ds.attributes

{'name': 'Tree cover loss 2021 (LM v3) STAGING',
 'slug': 'Tree-cover-loss-2021-LM-v3-STAGING',
 'type': None,
 'subtitle': None,
 'application': ['gfw'],
 'dataPath': None,
 'attributesPath': None,
 'connectorType': 'rest',
 'provider': 'gee',
 'userId': '622714c480348f001aab92b6',
 'connectorUrl': None,
 'sources': [],
 'tableName': 'projects/wri-datalab/HansenComposite_17',
 'status': 'failed',
 'published': True,
 'overwrite': False,
 'mainDateField': None,
 'env': 'staging',
 'geoInfo': False,
 'protected': False,
 'legend': {'date': [],
  'region': [],
  'country': [],
  'nested': [],
  'integer': [],
  'short': [],
  'byte': [],
  'double': [],
  'float': [],
  'half_float': [],
  'scaled_float': [],
  'boolean': [],
  'binary': [],
  'text': [],
  'keyword': []},
 'clonedHost': {},
 'errorMessage': None,
 'taskId': None,
 'createdAt': '2022-06-02T15:20:42.582Z',
 'updatedAt': '2022-06-02T15:21:13.909Z',
 'dataLastUpdated': None,
 'widget': [],
 'widgetRelevantProps': [],
 'laye

## Get associated layer id

In [11]:
ds.layers

[Layer 3c4225a4-a8d8-4d73-b12d-0c47dec84c76 Tree cover loss - 2001-2021]

In [12]:
lay = ds.layers[0]
lay

Layer 3c4225a4-a8d8-4d73-b12d-0c47dec84c76 Tree cover loss - 2001-2021

## Check properties in production  
Get a first view of the layer's structure in production. Also, check if the layer uses a decode function (only applies to raster data), update if necessary.

In [13]:
ds_prod.layers

[Layer 2de8aa6d-f523-481a-9ee5-787c4f2facc0 Tree cover loss - 2001-2021]

In [14]:
prod_lay = ds_prod.layers[0]

In [15]:
prod_lay.attributes['layerConfig']

{'decode_config': [{'default': '2001-01-01',
   'key': 'startDate',
   'required': True},
  {'default': '2021-12-31', 'key': 'endDate', 'required': True}],
 'decode_function': 'treeCoverLoss',
 'params_config': [{'default': 30, 'key': 'thresh', 'required': True}],
 'source': {'maxzoom': 12,
  'minzoom': 2,
  'tiles': ['https://tiles.globalforestwatch.org/umd_tree_cover_loss/v1.9/tcd_{thresh}/{z}/{x}/{y}.png'],
  'type': 'raster'},
 'timeline_config': {'dateFormat': 'YYYY',
  'interval': 'years',
  'speed': 250,
  'step': 1},
 'type': 'raster'}

Take note if necessary  
```"decode_function": " "```

## Update layer info
Some common updates include:  
- Update tile cache URL  
- Update legend info (text, colors, selectors)
- Update / check metadata  

Where to access to the information / parameters to update:  
- `attributes`: Layer name (title in the legend) and status (published: True/False)  
- `layerConfig`: tile cache URL, decoding function, max zoom, time/threshold selectors and parameter used  
- `legendConfig`: legend type and components (colors, categories), selector's sentence, extra text info  
- `interactionConfig`: layer selector parameters (when >1 layer on dataset), analytics tracking keys, metadata endpoint, layer slug id



**Check staging layer properties**

In [18]:
#Layer config attributes
lay.attributes['layerConfig']

{'decode_config': [{'default': '2001-01-01',
   'key': 'startDate',
   'required': True},
  {'default': '2021-12-31', 'key': 'endDate', 'required': True}],
 'decode_function': 'treeCoverLoss',
 'params_config': [{'default': 30, 'key': 'thresh', 'required': True}],
 'source': {'maxzoom': 12,
  'minzoom': 2,
  'tiles': ['https://tiles.globalforestwatch.org/umd_tree_cover_loss/v1.9/tcd_{thresh}/{z}/{x}/{y}.png'],
  'type': 'raster'},
 'timeline_config': {'dateFormat': 'YYYY',
  'interval': 'years',
  'speed': 250,
  'step': 1},
 'type': 'raster'}

In [17]:
# Legend config attributes
lay.attributes['legendConfig']

{'items': [{'dataMaxZoom': 12,
   'threshold': 30,
   'color': '#DC6C9A',
   'name': 'Tree cover loss',
   'source': '(annual, 30m, global, Hansen/UMD/Google/USGS/NASA)',
   'notes': ['Displaying loss with {thresh} canopy density.',
    'Tree cover loss is not always deforestation.']}],
 'type': 'basic'}

### Example of layer update  
The payload provided must include the whole object (all the parameters) of the category, in this case `layerConfig`. Any parameter / value can be modified as desired inside that (for example, provide a new tile cache URL / version)  
This operation requires the API token passed as `token` parameter.

In [ ]:
lay.update(update_params={'layerConfig':{'decode_config': [{'default': '2001-01-01',
   'key': 'startDate',
   'required': True},
  {'default': '2021-12-31', 'key': 'endDate', 'required': True}],
 'decode_function': 'treeCoverLoss',
 'params_config': [{'default': 30, 'key': 'thresh', 'required': True}],
 'source': {'maxzoom': 12,
  'minzoom': 2,
  'tiles': ['https://tiles.globalforestwatch.org/umd_tree_cover_loss/v1.9/tcd_{thresh}/{z}/{x}/{y}.png'],
  'type': 'raster'},
 'timeline_config': {'dateFormat': 'YYYY',
  'interval': 'years',
  'speed': 250,
  'step': 1},
 'type': 'raster'}
}, token=API_TOKEN)

## Changes at dataset level

### Publish / unpublish dataset

In [ ]:
ds.update(update_params={
    'published': True,
}, token=API_TOKEN)

### Update dataset metadata  
Dataset metadata contains the information being displayed in the contents index (left hand menu in the platform to select layers).  
Some key parameters are:  
- name: Name of the dataset displayed in the contents index  
- description: text in the information icon  
- citation: information displayed under the dataset name  
- color: toggle button color  

In [20]:
ds.metadata

Metadata 6298d5542e5b9a001a7f7a3a

In [21]:
metas = ds.metadata[0]
metas.attributes

{'dataset': '30ec7d3d-1047-4e11-a765-785cead8375a',
 'application': 'gfw',
 'resource': {'id': '30ec7d3d-1047-4e11-a765-785cead8375a', 'type': 'dataset'},
 'language': 'en',
 'info': {'citation': 'annual, 30m, global, Hansen/UMD/Google/USGS/NASA',
  'color': '#fe6598',
  'description': 'Identifies areas of gross tree cover loss.',
  'isLossLayer': True,
  'name': 'Tree cover loss'},
 'createdAt': '2022-06-02T15:20:52.595Z',
 'updatedAt': '2022-06-02T15:20:52.595Z',
 'status': 'published'}

**To update metadata, include the whole object as payload and modify as needed**  
**NOTE: to check the changes on the metadata, the dataset needs to be reloaded (create `ds` and `metas` variables again**)

In [ ]:
metas.update(update_params={'dataset': '30ec7d3d-1047-4e11-a765-785cead8375a',
 'application': 'gfw',
 'resource': {'id': '30ec7d3d-1047-4e11-a765-785cead8375a', 'type': 'dataset'},
 'language': 'en',
 'info': {'citation': 'annual, 30m, global, Hansen/UMD/Google/USGS/NASA',
  'color': '#fe6598',
  'description': 'Identifies areas of gross tree cover loss.',
  'isLossLayer': True,
  'name': 'Tree cover loss'},
 'createdAt': '2022-06-02T15:20:52.595Z',
 'updatedAt': '2022-06-02T15:20:52.595Z',
 'status': 'published'}, token = API_TOKEN)

## Deploy to production  


### 1) load staging and production datasets (if not already) via id  

In [ ]:
ds = lmi.Dataset('')
ds

In [ ]:
ds_prod = lmi.Dataset('')
ds_prod

### 2) Clone staging dataset to production  
Update dataset asset name.  
Change environment to `production` afterwards (it does not work during clonning)

In [ ]:
dataset_name = 'SET NAME (v202205)'


ds_clone = ds.clone(
    token=API_TOKEN,
    env='production',
    dataset_params={
        'name': dataset_name,
        'application': ['gfw']
    },
    clone_children=True
)
ds_clone.update(update_params={'env':'production'}, token=API_TOKEN)

### 3) Unpublish old production dataset  
Update name to indicate old dataset

In [ ]:
ds_prod.update(update_params={
    'name': 'SET NAME (DEPRECATED)',
    'published':False}, token=API_TOKEN)